In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV

from time import time
from datetime import datetime

import pickle
import os

from sklearn.metrics import mean_absolute_percentage_error

from sklearn.model_selection import KFold


from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.base import BaseEstimator, TransformerMixin
from patsy import bs, dmatrix
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from patsy import dmatrix
from statsmodels.nonparametric.smoothers_lowess import lowess
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import HuberRegressor



from sklearn.preprocessing import PolynomialFeatures
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, DotProduct, RBF
from pygam import LinearGAM, s, f, te
from sklearn.preprocessing import FunctionTransformer

from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from statsmodels.nonparametric.smoothers_lowess import lowess

from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasRegressor

from pptx import Presentation
from pptx.util import Inches, Pt
import os

import warnings
warnings.filterwarnings('ignore')




In [37]:
# %pip install keras
# %pip install tensorflow
# %pip install scikeras
# %pip install scikit-learn
# %pip install pygam
# %pip install catboost
# %pip install lightgbm
# %pip install xgboost
# %pip install statsmodels
# %pip install patsy
# %pip install pptx



In [38]:
global map_name
map_name = 'map2' ## map2 = talhão 2. map1 = talhão 1

# Compatibilidade com sklearn

In [39]:

class RBFFeatureTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, gamma=0.1):
        self.gamma = gamma

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.exp(-(X ** 2) / (2.0 * (self.gamma ** 2)))

def local_regression(X, y):
    smooth = lowess(y, X, frac=0.1)
    return smooth[:, 1]

class LocalRegressionTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.array([local_regression(X[:, i], X[:, -1]) for i in range(X.shape[1] - 1)]).T


class LinearGAMWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, **kwargs):
        self.model = LinearGAM(**kwargs)

    def fit(self, X, y):
        self.model.fit(X, y)
        return self

    def predict(self, X):
        return self.model.predict(X)

# Carregar dados

In [40]:
def load_data(file_path):
    """
    Load data from a CSV file into a Pandas DataFrame.

    Parameters:
    - file_path (str): Path to the CSV file.

    Returns:
    - pd.DataFrame: Loaded DataFrame containing the data.
    """
    data_frame = pd.read_csv(file_path)
    return data_frame

# Separar X e Y

In [41]:
def exclude_lat_and_long(df):
    """
    Exclude latitude and longitude columns from the DataFrame.

    Parameters:
    - df: DataFrame to be processed.

    Returns:
    - DataFrame: Processed DataFrame without latitude and longitude columns.
    """
    return df.drop(columns=['latitude', 'longitude'])


def filter_variables_by_name(df, exclude_coordinates=True):
    """
    Filter variables in the DataFrame based on the presence of '.tiff' in column names.

    Parameters:
    - df: DataFrame to be processed.
    - exclude_coordinates: Boolean indicating whether to exclude latitude and longitude columns.

    Returns:
    - Tuple of DataFrames: One containing '.tiff' variables and the other containing non-'.tiff' variables.
    """
    tiff_variables = [col for col in df.columns if '.tiff' in col]
    non_tiff_variables = [col for col in df.columns if '.tiff' not in col]

    tiff_df = df[tiff_variables]
    non_tiff_df = df[non_tiff_variables]

    if exclude_coordinates:
        non_tiff_df = exclude_lat_and_long(non_tiff_df)

    return tiff_df, non_tiff_df


def split_target(data, target_feature):
    """
    Split the DataFrame into feature (X) and target (y) variables.

    Parameters:
    - data: DataFrame to be split.
    - target_feature: Name of the target feature.

    Returns:
    - Tuple of DataFrames: Feature variables (X) and target variable (y).
    """
    X = data.drop(columns=[target_feature])
    y = data[target_feature]
    return X, y


def split_data(X, y, coords, split_method='slice', part_to_split=10):
    """
    Split the data into training and testing sets based on the specified split method.

    Parameters:
    - X: Feature variables.
    - y: Target variable.
    - coords: DataFrame containing latitude and longitude columns.
    - split_method: Method for data splitting ('quadrant', 'slice', or 'random').
    - part_to_split: Number of parts to split the data.

    Returns:
    - Tuple of DataFrames: Training and testing sets for feature variables and target variable.
    """
    if split_method == 'quadrant':
        return split_data_by_quadrant(X, y, coords, part_to_split)
    elif split_method == 'random':
        return split_data_randomly(X, y)
    elif split_method == 'slice':
        return split_data_by_slice(X, y, coords, part_to_split)
    else:
        raise ValueError("Invalid split method. Valid options are 'quadrant', 'slice', and 'random'.")


def divide_into_quadrants(df, lat_col, lon_col):
    """
    Divide the DataFrame into four quadrants based on the mean coordinates.

    Parameters:
    - df: DataFrame to be divided.
    - lat_col: Name of the latitude column.
    - lon_col: Name of the longitude column.

    Returns:
    - List of DataFrames: Four quadrants of the original DataFrame.
    """
    # Calculate the average of coordinates to find the central point
    lat_mean = df[lat_col].mean()
    lon_mean = df[lon_col].mean()

    # Divide the DataFrame into four quadrants
    quadrant_1 = df[(df[lat_col] >= lat_mean) & (df[lon_col] >= lon_mean)]
    quadrant_2 = df[(df[lat_col] >= lat_mean) & (df[lon_col] < lon_mean)]
    quadrant_3 = df[(df[lat_col] < lat_mean) & (df[lon_col] >= lon_mean)]
    quadrant_4 = df[(df[lat_col] < lat_mean) & (df[lon_col] < lon_mean)]

    quadrant_list = [quadrant_1, quadrant_2, quadrant_3, quadrant_4]
    # Return the three quadrants together and the separated quadrant
    return quadrant_list


def divide_into_slices(df, lat_col, lon_col, N):
    """
    Divide the DataFrame into N slices based on latitude and longitude columns.

    Parameters:
    - df: DataFrame to be divided.
    - lat_col: Name of the latitude column.
    - lon_col: Name of the longitude column.
    - N: Number of slices.

    Returns:
    - List of DataFrames: N slices of the original DataFrame.
    """
    # Check if N is a valid value
    if N <= 0:
        raise ValueError("N must be a positive integer.")

    # Sort the DataFrame based on latitude and longitude columns
    df_sorted = df.sort_values(by=[lat_col, lon_col])

    # Calculate the number of samples in each part
    num_samples = len(df_sorted)
    samples_per_part = num_samples // N
    remainder = num_samples % N

    parts = []

    # Divide the DataFrame into N parts
    start_idx = 0
    for i in range(N):
        end_idx = start_idx + samples_per_part + (1 if i < remainder else 0)
        current_part = df_sorted.iloc[start_idx:end_idx]
        parts.append(current_part)
        start_idx = end_idx

    return parts


def merge_df(X, y):
    """
    Merge feature variables (X) and target variable (y) into a single DataFrame.

    Parameters:
    - X: Feature variables.
    - y: Target variable.

    Returns:
    - DataFrame: Merged DataFrame containing both X and y.
    """
    return pd.concat([X, y], axis=1)


def split_data_by_slice(X, y, coords, slice_to_split=10):
    """
    Split the data into training and testing sets by dividing into slices.

    Parameters:
    - X: Feature variables.
    - y: Target variable.
    - coords: DataFrame containing latitude and longitude columns.
    - slice_to_split: Index of the slice to be used for testing.

    Returns:
    - Tuple of DataFrames: Training and testing sets for feature variables and target variable.
    """
    # Merge X and y
    new_data = merge_df(X, y)

    # Merge X and y with coordinates
    new_data = merge_df(new_data, coords)

    # Divide into slices
    slice_list = divide_into_slices(new_data, 'latitude', 'longitude', 10)

    # Get the slice to be used for testing
    test_slice = slice_list.pop(slice_to_split - 1)

    # Combine the other slices
    train_slices = combine_parts(slice_list)

    # Separate band and nutrient variables
    train_slice_band, train_slice_nutrients = filter_variables_by_name(train_slices)
    test_slice_band, test_slice_nutrients = filter_variables_by_name(test_slice)

    return train_slice_band, test_slice_band, train_slice_nutrients, test_slice_nutrients


def split_data_by_quadrant(X, y, coords, quadrant_to_split=4):
    """
    Split the data into training and testing sets by dividing into quadrants.

    Parameters:
    - X: Feature variables.
    - y: Target variable.
    - coords: DataFrame containing latitude and longitude columns.
    - quadrant_to_split: Index of the quadrant to be used for testing.

    Returns:
    - Tuple of DataFrames: Training and testing sets for feature variables and target variable.
    """
    # Merge X and y
    new_data = merge_df(X, y)

    # Merge X and y with coordinates
    new_data = merge_df(new_data, coords)

    # Divide into quadrants
    quadrant_list = divide_into_quadrants(new_data, 'latitude', 'longitude')

    # Get the quadrant to be used for testing
    test_quadrant = quadrant_list.pop(quadrant_to_split - 1)

    # Combine the other quadrants
    train_quadrants = combine_quadrants(quadrant_list)

    # Separate band and nutrient variables
    train_quadrant_band, train_quadrant_nutrients = filter_variables_by_name(train_quadrants)
    test_quadrant_band, test_quadrant_nutrients = filter_variables_by_name(test_quadrant)

    return train_quadrant_band, test_quadrant_band, train_quadrant_nutrients, test_quadrant_nutrients


def split_data_randomly(X, y, test_size=0.2):
    """
    Split the data into training and testing sets randomly.

    Parameters:
    - X: Feature variables.
    - y: Target variable.
    - test_size: Proportion of the data to include in the test split.

    Returns:
    - Tuple of DataFrames: Training and testing sets for feature variables and target variable.
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    return X_train, X_test, y_train, y_test


def combine_parts(part_list):
    """
    Combine multiple DataFrames into a single DataFrame.

    Parameters:
    - part_list: List of DataFrames to be combined.

    Returns:
    - DataFrame: Combined DataFrame.
    """
    # Check if at least one DataFrame is provided
    if not part_list or len(part_list) < 2:
        raise ValueError("At least two DataFrames are required for combination.")

    # Combine all DataFrames in the list into a single DataFrame
    combined_df = pd.concat(part_list, ignore_index=True)

    return combined_df

# Preprocessamento

In [42]:
def preprocessing_data(numerical_features, categorical_features):
    """
    Create a data preprocessor using a column transformer with separate pipelines for numerical and categorical features.

    Parameters:
    - numerical_features: List of numerical feature names.
    - categorical_features: List of categorical feature names.

    Returns:
    - preprocessor: ColumnTransformer for data preprocessing.
    """
    ## Handling categorical variables
    categorical_transformer = Pipeline(steps=[
        ('encoder', OneHotEncoder())
    ])

    ## Handling numerical variables
    numerical_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    ## Transforming the data
    preprocessor = ColumnTransformer(transformers=[
        ('numbers', numerical_transformer, numerical_features),
        ('categories', categorical_transformer, categorical_features)
    ])

    ## Returning the data processor
    return preprocessor

# Modelos

In [43]:
from keras.optimizers import Adam, SGD, RMSprop
from keras.metrics import MeanAbsolutePercentageError, RootMeanSquaredError, MeanSquaredError, MeanAbsoluteError
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.metrics import MeanAbsoluteError


def get_hidden_layers(input_neurons, output_neurons):
    """
    Get the number of hidden layers for a neural network based on the number of input and output neurons.

    Parameters:
    - input_neurons: Number of input neurons.
    - output_neurons: Number of output neurons.

    Returns:
    - int: Number of hidden layers.
    """
    # Calculate the number of hidden layers based on the number of input and output neurons
    hidden_layers = int(np.log2(input_neurons * output_neurons))

    return hidden_layers

# Função para construir o modelo de rede neural
# def build_custom_neural_network(optimizer='adam', loss='mse', learning_rate=0.001, input_neurons=None, output_neurons=None, num_hidden_layers=1, neurons_per_layer=None):
#     if optimizer == 'adam':
#         optimizer = Adam(learning_rate=learning_rate)
#     elif optimizer == 'sgd':
#         optimizer = SGD(learning_rate=learning_rate)
#     elif optimizer == 'rmsprop':
#         optimizer = RMSprop(learning_rate=learning_rate)
    
#     model = Sequential()
    
#     # Adicionando a camada de entrada
#     model.add(Dense(input_neurons, input_dim=input_neurons, activation='relu'))
    
#     # Adicionando camadas ocultas
#     if neurons_per_layer is None:
#         neurons_per_layer = [input_neurons // (2 ** i) for i in range(num_hidden_layers)]
    
#     for neurons in neurons_per_layer:
#         model.add(Dense(neurons, activation='relu'))
    
#     # Adicionando a camada de saída
#     model.add(Dense(output_neurons, activation='linear'))  # Utilize 'linear' se for um problema de regressão
    
#     # Lista de métricas adicionais
#     metrics = [MeanAbsolutePercentageError(), RootMeanSquaredError(), MeanSquaredError(), MeanAbsoluteError()]
    
#     # Compilando o modelo com métricas adicionais
#     model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
#     return model

def build_simple_neural_network(input_neurons, output_neurons, optimizer='adam', loss='mse'):
    """
    Build a simple neural network model with one hidden layer.

    Parameters:
    - input_neurons: Number of input neurons.
    - output_neurons: Number of output neurons.
    - optimizer: Optimizer for the model.
    - loss: Loss function for the model.
    
    Returns:
    - Sequential: Compiled neural network model.
    """
    
    model = Sequential()
    
    # Adicionando a camada de entrada
    model.add(Dense(input_neurons, input_dim=input_neurons, activation='relu'))
    
    # Adicionando a camada oculta com o mesmo número de neurônios da camada de entrada
    model.add(Dense(input_neurons, activation='relu'))
    
    # Adicionando a camada de saída
    model.add(Dense(output_neurons, activation='linear'))  # Utilize 'linear' se for um problema de regressão
    
    # Compilando o modelo
    model.compile(optimizer=optimizer, loss=loss, metrics=[MeanSquaredError()])
    
    return model
    
def create_models_pipeline(preprocessor, grid, neural_network = False, input_layer_neurons=1, output_layer_neurons=1, n_folds=10):
    """
    Create a pipeline of models, with or without hyperparameter tuning using grid search.

    Parameters:
    - preprocessor: Data preprocessor for feature transformation.
    - grid: Boolean indicating whether to use grid search for hyperparameter tuning.
    - n_folds: Number of folds for cross-validation.
    - neural_network: Boolean indicating whether to include a neural network model.

    Returns:
    - models_pipeline: Dictionary containing models as values, with or without grid search.
    """
    models_pipeline = create_models(preprocessor, neural_network, input_layer_neurons, output_layer_neurons)

    if grid:
        grid_models_pipeline = create_grid_models(models_pipeline, n_folds, input_layer_neurons, output_layer_neurons)
        return grid_models_pipeline

    return models_pipeline


def create_models(preprocessor, neural_network = False, input_layer_neurons = 1, output_layer_neurons = 1):
    """
    Create a dictionary of models with associated pipelines for training.

    Parameters:
    - preprocessor: Data preprocessor for feature transformation.
    - neural_network: Boolean indicating whether to include a neural network model.
    - input_layer_neurons: Number of input neurons for the neural network model.
    - output_layer_neurons: Number of output neurons for the neural network model.

    Returns:
    - models: Dictionary containing machine learning models with associated pipelines.
    """

    if neural_network:
        models = {
            'NeuralNetwork': KerasRegressor(
                build_fn=build_simple_neural_network,
                input_neurons=input_layer_neurons,
                output_neurons=output_layer_neurons,
                epochs=100,
                batch_size=32,
                verbose=0
            )
        }

        # Create pipelines for each model
        for model_name, model in models.items():
            models[model_name] = Pipeline(steps=[
                ('preprocessor', preprocessor),
                ('dim_red', PCA(5)),
                ('model', model)
            ])

        return models
        
    else:
        models = {
            'LinearRegression': LinearRegression(),
            # 'RandomForest': RandomForestRegressor(),
            # 'SVR': SVR(),
            # 'Ridge': Ridge(),
            # 'Lasso': Lasso(),
            # 'CatBoost': CatBoostRegressor(),
            # 'LGBM': LGBMRegressor(),
            # 'XGBoost': XGBRegressor(),
            # 'BayesianRidge': BayesianRidge(),
            # 'KNeighbors': KNeighborsRegressor(),
            # 'GradientBoosting': GradientBoostingRegressor(),
            # 'MLP': MLPRegressor(),
            # 'ElasticNet': ElasticNet(),
            # 'HuberRegressor': HuberRegressor(),
            # 'GAM': LinearGAMWrapper(),
        }

        # Create pipelines for each model
        for model_name, model in models.items():
            models[model_name] = Pipeline(steps=[
                ('preprocessor', preprocessor),
                ('dim_red', PCA()),
                ('model', model)
            ])

        # Special case for Polynomial Regression
        # models['PolynomialRegression'] = Pipeline(steps=[
        #     ('preprocessor', preprocessor),
        #     ('poly', PolynomialFeatures(degree=2)),
        #     ('dim_red', PCA()),
        #     ('model', LinearRegression())
        # ])
        
        
        # models['BasisFunctions'] = Pipeline(steps=[
        #     ('preprocessor', preprocessor),
        #     ('basis', RBFFeatureTransformer()),
        #     ('dim_red', PCA()),
        #     ('model', LinearRegression())
        # ])

        # models['LocalRegression'] = Pipeline(steps=[
        #     ('preprocessor', preprocessor),
        #     ('local', LocalRegressionTransformer()),
        #     ('dim_red', PCA()),
        #     ('model', LinearRegression())
        # ])

        return models



# Otimização

In [44]:
def create_grid_models(models_pipeline, n_folds = 10 , input_layer_neurons = 1, output_layer_neurons = 1):
    """
    Create a dictionary of GridSearchCV objects with hyperparameter grids for each model.

    Parameters:
    - models_pipeline: Dictionary containing machine learning models to be tuned.
    - n_folds: Number of folds for cross-validation.
    - input_layer_neurons: Number of input neurons for the neural network model.
    - output_layer_neurons: Number of output neurons for the neural network model.
    

    Returns:
    - grid_models: Dictionary containing GridSearchCV objects for each model.
    """

    grid_models = {}

    grid_params = {
        'LinearRegression': {
            'dim_red__n_components': [5],
            # 'model__fit_intercept': [True, False],
            # 'model__normalize': [True, False],
            # 'model__copy_X': [True, False],
        },
        'RandomForest': {
            'dim_red__n_components': [5],
            # 'model__n_estimators': [100, 200, 300, 1000],
            # 'model__max_depth': [None, 5, 10, 20],
            # 'model__min_samples_split': [2, 5, 10],
            # 'model__min_samples_leaf': [1, 2, 4],
            # 'model__max_features': ['auto', 'sqrt', 'log2'],
            # 'model__bootstrap': [True, False],
        },
        'SVR': {
            'dim_red__n_components': [5],
            # 'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            # 'model__degree': [3, 5, 7],
            # 'model__gamma': ['scale', 'auto'],
            # 'model__coef0': [0.0, 1.0, 2.0],
            # 'model__tol': [0.001, 0.01, 0.1],
            # 'model__C': [1.0, 2.0, 3.0],
            # 'model__epsilon': [0.1, 0.2, 0.3],
            # 'model__shrinking': [True, False],
            # 'model__cache_size': [200, 500, 1000],
            # 'model__verbose': [True, False],
            # 'model__max_iter': [-1],
        },
        'GAM': {
            'dim_red__n_components': [5],
        #     'model__terms': [[s(0), s(1), s(2)]],
        #     'model__n_splines': [10, 20, 30],
        #     'model__lam': [[0.6], [0.8], [1.0]],  # Ajustado para uma lista aninhada
        #     'model__tol': [0.001, 0.01, 0.1],
        #     'model__max_iter': [100, 200, 300],
        #     'model__scale': [None],        
        },
        'Ridge': {
            'dim_red__n_components': [5],
            # 'model__alpha': [0.1, 1.0, 2.0],
            # 'model__fit_intercept': [True, False],
            # 'model__copy_X': [True, False],
            # 'model__max_iter': [None],
            # 'model__tol': [0.001, 0.01, 0.1],
            # 'model__solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
            # 'model__random_state': [None],
        },
        'Lasso': {
            'dim_red__n_components': [5],
            # 'model__alpha': [0.1, 1.0, 2.0],
            # 'model__fit_intercept': [True, False],
            # 'model__precompute': [True, False],
            # 'model__copy_X': [True, False],
            # 'model__max_iter': [1000],
            # 'model__tol': [0.0001, 0.001, 0.01],
            # 'model__warm_start': [True, False],
            # 'model__positive': [True, False],
            # 'model__selection': ['cyclic', 'random'],
            # 'model__random_state': [None],
        },
        'CatBoost': {
            'dim_red__n_components': [5],
            # 'model__iterations': [100, 200, 300],
            # 'model__learning_rate': [0.03, 0.1, 0.3],
            # 'model__depth': [4, 6, 10],
            # 'model__l2_leaf_reg': [1, 3, 5, 7, 9],
            # 'model__border_count': [32, 5, 10, 20, 50, 100, 200],
            # 'model__thread_count': [4],
            # 'model__random_state': [None],
        },
        'LGBM': {
            'dim_red__n_components': [5],
            # 'model__boosting_type': ['gbdt', 'dart', 'goss', 'rf'],
            # 'model__num_leaves': [31, 50, 100],
            # 'model__max_depth': [-1, 5, 10, 20],
            # 'model__learning_rate': [0.001, 0.01, 0.1],
            # 'model__n_estimators': [100, 200, 300],
            # 'model__subsample_for_bin': [200000, 500000, 1000000],
            # 'model__objective': ['regression', 'regression_l1', 'mape'],
            # 'model__class_weight': [None],
            # 'model__min_split_gain': [0.0],
            # 'model__min_child_weight': [0.001],
            # 'model__min_child_samples': [20],
            # 'model__subsample': [1.0],
            # 'model__subsample_freq': [0],
            # 'model__colsample_bytree': [1.0],
            # 'model__reg_alpha': [0.0],
            # 'model__reg_lambda': [0.0],
            # 'model__random_state': [None],
            # 'model__n_jobs': [-1],
            # 'model__silent': [True],
            # 'model__importance_type': ['split', 'gain'],
        },
        'XGBoost': {
            'dim_red__n_components': [5],
            # 'model__max_depth': [3, 5, 10],
            # 'model__learning_rate': [0.001, 0.01, 0.1],
            # 'model__n_estimators': [100, 200, 300],
            # 'model__verbosity': [0],
            # 'model__silent': [None],
            # 'model__objective': ['reg:linear', 'reg:gamma', 'reg:tweedie'],
            # 'model__booster': ['gbtree', 'gblinear', 'dart'],
            # 'model__n_jobs': [-1],
            # 'model__gamma': [0.0],
            # 'model__min_child_weight': [1.0],
            # 'model__max_delta_step': [0.0],
            # 'model__subsample': [1.0],
            # 'model__colsample_bytree': [1.0],
            # 'model__colsample_bylevel': [1.0],
            # 'model__colsample_bynode': [1.0],
            # 'model__reg_alpha': [0.0],
            # 'model__reg_lambda': [1.0],
            # 'model__scale_pos_weight': [1.0],
            # 'model__base_score': [0.5],
            # 'model__random_state': [None],
            # 'model__missing': [None],
            # 'model__importance_type': ['gain', 'weight', 'cover', 'total_gain', 'total_cover'],
        },
        'BayesianRidge': {
            'dim_red__n_components': [5],
            # 'model__n_iter': [300, 500, 1000],
            # 'model__tol': [0.001, 0.01, 0.1],
            # 'model__alpha_1': [1e-06, 1e-05, 1e-04],
            # 'model__alpha_2': [1e-06, 1e-05, 1e-04],
            # 'model__lambda_1': [1e-06, 1e-05, 1e-04],
            # 'model__lambda_2': [1e-06, 1e-05, 1e-04],
            # 'model__compute_score': [False],
            # 'model__fit_intercept': [True, False],
            # 'model__normalize': [True, False],
            # 'model__copy_X': [True, False],
            # 'model__verbose': [False],
        },
        'KNeighbors': {
            'dim_red__n_components': [5],
            # 'model__n_neighbors': [5, 10, 20],
            # 'model__weights': ['uniform', 'distance'],
            # 'model__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
            # 'model__leaf_size': [30],
            # 'model__p': [1, 2],
            # 'model__metric': ['minkowski'],
            # 'model__metric_params': [None],
            # 'model__n_jobs': [None],
        },
        'GradientBoosting': {
            'dim_red__n_components': [5],
            # 'model__loss': ['ls', 'lad', 'huber', 'quantile'],
            # 'model__learning_rate': [0.001, 0.01, 0.1],
            # 'model__n_estimators': [100, 200, 300],
            # 'model__subsample': [1.0],
            # 'model__criterion': ['friedman_mse', 'mse', 'mae'],
            # 'model__min_samples_split': [2, 5, 10],
            # 'model__min_samples_leaf': [1, 2, 4],
            # 'model__min_weight_fraction_leaf': [0.0],
            # 'model__max_depth': [3, 5, 10],
            # 'model__min_impurity_decrease': [0.0],
            # 'model__min_impurity_split': [None],
            # 'model__init': [None],
            # 'model__random_state': [None],
            # 'model__max_features': [None],
            # 'model__alpha': [0.9],
            # 'model__verbose': [0],
            # 'model__max_leaf_nodes': [None],
            # 'model__warm_start': [False],
            # 'model__presort': ['deprecated'],
            # 'model__validation_fraction': [0.1],
            # 'model__n_iter_no_change': [None],
            # 'model__tol': [0.0001],
            # 'model__ccp_alpha': [0.0],
        },
        'MLP': {
            'dim_red__n_components': [5],
            # 'model__hidden_layer_sizes': [(100,), (50, 50), (50, 50, 50)],
            # 'model__activation': ['identity', 'logistic', 'tanh', 'relu'],
            # 'model__solver': ['lbfgs', 'sgd', 'adam'],
            # 'model__alpha': [0.0001, 0.001, 0.01],
            # 'model__batch_size': ['auto'],
            # 'model__learning_rate': ['constant', 'invscaling', 'adaptive'],
            # 'model__learning_rate_init': [0.001],
            # 'model__power_t': [0.5],
            # 'model__max_iter': [200, 500, 1000],
            # 'model__shuffle': [True],
            # 'model__random_state': [None],
            # 'model__tol': [0.0001],
            # 'model__verbose': [False],
            # 'model__warm_start': [False],
            # 'model__momentum': [0.9],
            # 'model__nesterovs_momentum': [True],
            # 'model__early_stopping': [False],
            # 'model__validation_fraction': [0.1],
            # 'model__beta_1': [0.9],
            # 'model__beta_2': [0.999],
            # 'model__epsilon': [1e-08],
            # 'model__n_iter_no_change': [10],
            # 'model__max_fun': [15000],
        },
        'ElasticNet': {
            'dim_red__n_components': [5],
            # 'model__alpha': [0.1, 1.0, 2.0],
            # 'model__l1_ratio': [0.1, 0.5, 0.9],
            # 'model__fit_intercept': [True, False],
            # 'model__normalize': [True, False],
            # 'model__precompute': [True, False],
            # 'model__max_iter': [1000],
            # 'model__copy_X': [True, False],
            # 'model__tol': [0.0001, 0.001, 0.01],
            # 'model__warm_start': [True, False],
            # 'model__positive': [True, False],
            # 'model__random_state': [None],
            # 'model__selection': ['cyclic', 'random'],
        },
        'HuberRegressor': {
            'dim_red__n_components': [5],
            # 'model__epsilon': [1.35, 1.5, 1.75],
            # 'model__max_iter': [100, 200, 300],
            # 'model__alpha': [0.0001, 0.001, 0.01],
            # 'model__warm_start': [False],
        },    
        'PolynomialRegression': {
            'dim_red__n_components': [5],
            # 'poly__degree': [2, 3, 4],
        },
        'SplineRegression': {
            'dim_red__n_components': [5],
            # 'model__n_splines': [10, 20, 30],
            # 'model__lam': [0.6, 0.8, 1.0],
            # 'model__tol': [0.001, 0.01, 0.1],
            # 'model__max_iter': [100, 200, 300],
            # 'model__fit_intercept': [True, False],
            # 'model__scale': [None],
        },
        'SmoothingSplines': {
            'dim_red__n_components': [5],
            # 'model__n_splines': [10, 20, 30],
            # 'model__lam': [0.6, 0.8, 1.0],
            # 'model__tol': [0.001, 0.01, 0.1],
            # 'model__max_iter': [100, 200, 300],
            # 'model__fit_intercept': [True, False],
            # 'model__scale': [None],
        },
        'StepFunctions': {
            'dim_red__n_components': [5],
            # 'model__n_splines': [10, 20, 30],
            # 'model__lam': [0.6, 0.8, 1.0],
            # 'model__tol': [0.001, 0.01, 0.1],
            # 'model__max_iter': [100, 200, 300],
            # 'model__fit_intercept': [True, False],
            # 'model__scale': [None],
        },
        'BasisFunctions': {
            'dim_red__n_components': [5],
            # 'model__gamma': [0.1, 0.5, 1.0],
        },
        'LocalRegression': {
            'dim_red__n_components': [5],
        },
        'NeuralNetwork': {
            'model__optimizer': ['adam'],
            'model__input_neurons': [input_layer_neurons,],  # Formato da entrada da rede neural
            'model__output_neurons': [output_layer_neurons],  # Formato da saída da rede neural
            'model__epochs': [100],
            'model__loss': ['mse'],
            'model__batch_size': [32]
        }
    }

    # Create GridSearchCV objects for each model
    for model_name, model in models_pipeline.items():
        grid_models[model_name] = GridSearchCV(model, param_grid = grid_params[model_name], cv = KFold(n_splits=n_folds, shuffle=False), scoring = 'neg_mean_squared_error', return_train_score=True)


    return grid_models


# Treino

In [45]:


def log_CV_scores(model_name, grid_result):
    """
    Log the best parameters and score of a model from grid search results.

    Parameters:
    - model_name: Name of the model.
    - grid_result: Result of grid search.

    Returns:
    - None
    """
    best_params = grid_result.best_params_
    best_score = grid_result.best_score_

    print(f'Best parameters of {model_name} are: {best_params}')
    print(f'Best score of {model_name} is: {best_score}\n')

    mean_cv_score = grid_result.cv_results_['mean_test_score'][0]
    std_cv_score = grid_result.cv_results_['std_test_score'][0]

    print(f'Mean CVs score: {mean_cv_score}')
    print(f'Stds CVs score: {std_cv_score}\n')


def train_models(X, y, models_pipeline, grid):
    """
    Train machine learning models using a specified pipeline.

    Parameters:
    - X: Feature matrix of the dataset.
    - y: Target variable of the dataset.
    - models_pipeline: Dictionary containing machine learning models to be trained.
    - grid: Boolean indicating whether to perform grid search.

    Returns:
    - models: Dictionary containing trained machine learning models.
    - scores: Dictionary containing training scores for each model.
    """
    models = {}
    scores = {}

    # Loop through each model in the pipeline
    for name, model in models_pipeline.items():
        print(f'Training {name}')

        # Train the model with or without grid search
        if grid:
            grid_result = model.fit(X, y)

            # Extract and log training metrics from grid search results
            train_metrics = {
                'Mean MSE': abs(grid_result.cv_results_['mean_test_score'][0]),
                'Std MSE': abs(grid_result.cv_results_['std_test_score'][0]),
            }
            
            log_CV_scores(name, grid_result)

            # Save the best estimator in the models dictionary
            models[name] = grid_result.best_estimator_

            # Store training metrics in the scores dictionary
            scores[name] = train_metrics

        else:
            models[name] = model.fit(X, y)

        print()

    # Return the trained models and their training scores
    return models, scores

# Teste

In [46]:

def evaluate_models(X, y, models, neural_network=False):
    """
    Evaluate the performance of machine learning models on a given dataset.

    Parameters:
    - X: Feature matrix of the dataset.
    - y: Target variable of the dataset.
    - models: Dictionary containing machine learning models.
    - neural_network: Boolean indicating whether the models are neural networks.

    Returns:
    - models_scores: Dictionary containing evaluation scores for each model.
    """
    models_scores = {}  # Dictionary to store evaluation scores for each model
    scores_df = pd.DataFrame()  # Initialize an empty DataFrame for the scores

    # Loop through each model
    for name, model in models.items():
        # Make predictions using the model
        predictions = model.predict(X)

        # Calculate evaluation scores
        if not neural_network:
            scores = calculate_non_neural_scores(y, predictions)
            models_scores[name] = scores
        else:
            scores_df = calculate_neural_scores(y, predictions, name, scores_df)

    if neural_network:
        return scores_df
    else:
        return models_scores

def calculate_non_neural_scores(y_true, y_pred):
    """
    Calculate evaluation scores for non-neural network models.

    Parameters:
    - y_true: True target values.
    - y_pred: Predicted values.

    Returns:
    - scores: Dictionary containing evaluation scores.
    """
    scores = {
        'MSE': mean_squared_error(y_true, y_pred),
        'MAE': mean_absolute_error(y_true, y_pred),
        'R2': r2_score(y_true, y_pred),
        'MAPE': mean_absolute_percentage_error(y_true, y_pred)
    }
    return scores

def calculate_neural_scores(y_true, y_pred, name, scores_df):
    """
    Calculate evaluation scores for neural network models.

    Parameters:
    - y_true: True target values.
    - y_pred: Predicted values.
    - name: Name of the model.
    - scores_df: DataFrame containing evaluation scores for each model.

    Returns:
    - scores_df: Updated DataFrame containing evaluation scores for each model.
    """
    model_scores = {}
    for i, nutrient in enumerate(y_true.columns):
        nutrient_scores = {
            'MSE': mean_squared_error(y_true[nutrient], y_pred[:, i]),
            'MAE': mean_absolute_error(y_true[nutrient], y_pred[:, i]),
            'R2': r2_score(y_true[nutrient], y_pred[:, i]),
            'MAPE': mean_absolute_percentage_error(y_true[nutrient], y_pred[:, i])
        }
        model_scores[nutrient] = nutrient_scores
    
    model_df = pd.DataFrame(model_scores).stack().unstack(0)
    model_df.columns = pd.MultiIndex.from_product([[name], model_df.columns])
    
    scores_df = pd.concat([scores_df, model_df], axis=1)

    return scores_df

# Salvando resultados

In [47]:

def save_results(df, data_split_method, part_to_split, grid, neural_network):
    """
    Save evaluation results DataFrame to an Excel file with a timestamped filename.

    Parameters:
    - df: DataFrame containing evaluation results.
    - data_split_method: Method used for data splitting.
    - part_to_split: Number of parts to split the data.
    - grid: Boolean indicating whether a grid search was performed.

    Returns:
    - None
    """
    now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

    filename = create_filename(data_split_method, part_to_split, grid, now, neural_network)

    df.to_excel(filename)

    print(f'Results saved at {filename}')

def create_filename(data_split_method, part_to_split, grid, timestamp, neural_network):
    """
    Create a filename based on the parameters.

    Parameters:
    - data_split_method: Method used for data splitting.
    - part_to_split: Number of parts to split the data.
    - grid: Boolean indicating whether a grid search was performed.
    - timestamp: Timestamp for the filename.

    Returns:
    - filename: Formatted filename.
    """
    if neural_network:
        filename = f'data/{map_name}/prediction_results/neural_network/neural_network_{timestamp}'
    else:
        filename = f'data/{map_name}/prediction_results/all_models/all_models_{timestamp}'

    if data_split_method in ('quadrant', 'slice'):
        filename += f'_part_splited_{part_to_split}'
    filename += '_results.xlsx'

    return filename


def create_directory_if_not_exists(directory):
    """
    Create a directory if it does not exist.

    Parameters:
    - directory: Path to the directory to be created.

    Returns:
    - None
    """
    # Check if the directory exists; if not, create it
    if not os.path.exists(directory):
        os.makedirs(directory)


def save_models(models, data_split_method, target_column, neural_network=False):
    """
    Save trained models to pickle files with timestamped filenames.

    Parameters:
    - models: Dictionary containing model names as keys and corresponding trained models as values.
    - data_split_method: Method used for data splitting.
    - target_column: Target column for which models were trained.
    - neural_network: Boolean indicating whether the models are neural networks.

    Returns:
    - None
    """
    # Get the current timestamp
    now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

    # Determine the directory based on the data splitting method and neural network mode
    if neural_network:
        directory = f'models/{map_name}/neural_network/{target_column}'
    elif data_split_method == 'quadrant':
        directory = f'models/{map_name}/quadrant/{target_column}'
    elif data_split_method == 'slice':
        directory = f'models/{map_name}/slice/{target_column}'
    else:
        directory = f'models/{map_name}/random/{target_column}'

    # Create the directory if it does not exist
    create_directory_if_not_exists(directory)

    # Save each model to a pickle file with a timestamped filename
    for model_name, model in models.items():
        model_filename = f'{directory}/{model_name}_{now}.pkl'
        with open(model_filename, 'wb') as model_file:
            pickle.dump(model, model_file)

        # Print a message indicating the saved model and its location
        print(f'Model {model_name} saved at {model_filename}')

# Pipeline

In [48]:
def pipeline(X, y, coords, numerical_features, categorical_features, grid, neural_network, data_split_method, part_to_split):
    """
    Execute the machine learning pipeline, including data preprocessing, model training, and evaluation.

    Parameters:
    - X (pd.DataFrame): Features dataset.
    - y (pd.Series): Target variable.
    - coords (pd.DataFrame): Coordinates dataset.
    - numerical_features (list): List of numerical features.
    - categorical_features (list): List of categorical features.
    - grid (bool): Flag indicating whether to perform grid search for hyperparameter tuning.
    - data_split_method (str): Method for splitting the data ('quadrant', 'slice', or 'random').
    - part_to_split (int): Number of parts to split the data.
    - neural_network (bool): Flag indicating whether to use a neural network.

    Returns:
    - dict: Validation scores for each trained model.
    - dict: Training metrics for each trained model.
    """
    np.random.seed(42)

    X_train, X_test, y_train, y_test = split_data(X, y, coords, data_split_method, part_to_split)

    preprocessor = preprocessing_data(numerical_features, categorical_features)

    models_pipeline = create_models_pipeline(preprocessor = preprocessor, 
                                             grid = grid, 
                                             neural_network = neural_network,
                                             input_layer_neurons = 5,#X_train.shape[1],
                                             output_layer_neurons = y_train.shape[1], 
                                             n_folds=10 if data_split_method == 'slice' else 5)
    
    print("Training models")

    trained_models, training_score = train_models(X_train, y_train, models_pipeline, grid)

    print("Evaluating models")

    validation_score = evaluate_models(X_test, y_test, trained_models, neural_network)

    print("Done")

    save_models(trained_models, data_split_method, y.name if not neural_network else y.columns.values, neural_network)

    # print(training_score)

    return validation_score, training_score

# Transformando em Slides

In [49]:
def save_metric_per_row(df, save_dir, subcolumn='MAPE', title='Column'):
    """
    Save bar charts for a specific metric (subcolumn) for each row in the DataFrame.

    Parameters:
    - df (pd.DataFrame): DataFrame with the metrics.
    - save_dir (str): Directory to save the bar charts.
    - subcolumn (str): Subcolumn within the DataFrame to visualize.
    - title (str): Base title for the charts.

    Returns:
    - None
    """
    # Check if the saving directory exists; create it if not
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # Get the model names from the DataFrame columns
    model_names = df.columns.levels[0]
    
    # Iterate over the rows of the DataFrame
    for index, row in df.iterrows():
        # Create a new directory for each index
        index_dir = os.path.join(save_dir, str(index))
        os.makedirs(index_dir, exist_ok=True)
        
        # Get the specified subcolumn values for the current row
        subcolumn_values = row.xs(subcolumn, level=1)
        
        # Sort the values and corresponding model names
        sorted_values, sorted_indices = zip(*sorted(zip(subcolumn_values, model_names)))
        
        # Increase the figure size to ensure enough space for x-axis labels
        plt.figure(figsize=(10, 6))
        
        # Create and save the bar chart for the sorted subcolumn values of the current row
        plt.bar(np.arange(len(model_names)), sorted_values, width=0.2)
        plt.ylabel(subcolumn)
        
        # Add a customized title (using the index as the title)
        title_with_index = f'{title} - {index}'
        plt.title(title_with_index)
        plt.xticks(np.arange(len(model_names)), sorted_indices, rotation=45, ha='right')
        
        # Adjust the bottom margin to ensure full visibility of x-axis labels
        plt.subplots_adjust(bottom=0.3)
        
        # Save the figure in the specific directory for the index
        file_path = os.path.join(index_dir, f'{index}.png')  # Use the title as the file name
        plt.savefig(file_path)
        plt.close()

def add_simple_slide(prs, target_column, model_name, mse_test_path, mse_training_path, mape_path):
    """
    Adds a slide to a PowerPoint presentation (pptx) containing three graphs: MSE Test, MSE Training, and MAPE
    for a specific target column and model.

    Parameters:
    - prs: PowerPoint presentation (Presentation) where the slide will be added.
    - target_column: Name of the target column.
    - model_name: Name of the model.
    - mse_test_path: Path to the MSE Test graph.
    - mse_training_path: Path to the MSE Training graph.
    - mape_path: Path to the MAPE graph.

    Returns:
    - None
    """
    # Choose a blank slide layout
    blank_slide_layout = prs.slide_layouts[6]

    # Add slide with the chosen layout
    slide = prs.slides.add_slide(blank_slide_layout)

    # Add title
    title_text = f"Analysis for column: {target_column}\nModel: {model_name}"
    add_text_box(slide, title_text, left=Inches(0), top=Inches(0), width=Inches(10), height=Inches(1))

    # Add graphs
    add_graph(slide, mse_test_path, top=Inches(2), left=Inches(0.1))
    add_graph(slide, mse_training_path, top=Inches(2), left=Inches(3))
    add_graph(slide, mape_path, top=Inches(2), left=Inches(6))

def add_text_box(slide, text, left, top, width, height):
    """Adds a text box to a slide."""
    txBox = slide.shapes.add_textbox(left=left, top=top, width=width, height=height)
    text_frame = txBox.text_frame
    p = text_frame.add_paragraph()
    p.text = text
    p.font.size = Pt(18)
    p.font.bold = True

def add_graph(slide, image_path, top, left, width=Inches(3.2), height=Inches(3.2)):
    """Adds a graph to a slide."""
    slide.shapes.add_picture(image_path, left, top, width=width, height=height)

def create_simple_slides(mse_test_images_dirs, mse_training_images_dirs, mape_images_dirs, target_columns, output_pptx):
    """
    Creates slides in a PowerPoint presentation (pptx) for the analysis of MSE Test, MSE Training, and MAPE
    for specific target columns and models.

    Parameters:
    - mse_test_images_dirs: Dictionary with model names as keys and directories containing MSE Test images as values.
    - mse_training_images_dirs: Dictionary with model names as keys and directories containing MSE Training images as values.
    - mape_images_dirs: Dictionary with model names as keys and directories containing MAPE images as values.
    - target_columns: List of target columns.
    - output_pptx: Path to the output pptx file.

    Returns:
    - None
    """
    # Initialize the presentation
    prs = Presentation()

    # Add slides for each column and model
    for target_column in target_columns:
        for model_name in mse_test_images_dirs:  # Assuming mse_test_images_dirs contains only the desired model directories
            # Find paths to the images
            mse_test_path = os.path.join(mse_test_images_dirs[model_name], target_column, f'{target_column}.png')
            mse_training_path = os.path.join(mse_training_images_dirs[model_name], target_column, f'{target_column}.png')
            mape_path = os.path.join(mape_images_dirs[model_name], target_column, f'{target_column}.png')

            # Add slide with the three images
            add_simple_slide(prs, target_column, model_name, mse_test_path, mse_training_path, mape_path)

    # Save the presentation
    prs.save(output_pptx)


def create_files_results(train_scores, test_scores, target_columns, neural_network=False):
    """
    Create image files and a PowerPoint presentation from training and test scores.

    Parameters:
    - train_scores (pd.DataFrame): DataFrame containing training scores.
    - test_scores (pd.DataFrame): DataFrame containing test scores.
    - target_columns (list): List of target columns.
    - neural_network (bool): Flag indicating whether the model is a neural network. Default is False.

    Returns:
    - None
    """

    # Define the base directory for saving images and slides
    base_dir = f'images/{map_name}/metrics'
    if neural_network:
        base_dir = f'{base_dir}/neural_network'
    else:
        base_dir = f'{base_dir}/all_models'

    # Define directories for saving images
    test_metrics_dir = f'{base_dir}/test_metrics'
    training_metrics_dir = f'{base_dir}/training_metrics'

    test_mse_dir = f'{test_metrics_dir}/mse'
    test_mape_dir = f'{test_metrics_dir}/mape'
    test_mae_dir = f'{test_metrics_dir}/mae'

    train_mean_mse_dir = f'{training_metrics_dir}/mean'
    train_std_mse_dir = f'{training_metrics_dir}/std'

    # Create images results from TEST SCORES
    save_metric_per_row(test_scores, save_dir=test_mape_dir, subcolumn='MAPE', title='Test Scores - MAPE')
    save_metric_per_row(test_scores, save_dir=test_mse_dir, subcolumn='MSE', title='Test Scores - MSE')
    save_metric_per_row(test_scores, save_dir=test_mae_dir, subcolumn='MAE', title='Test Scores - MAE')

    # Create images results from TRAINING SCORES if not a neural network
    if not neural_network:
        save_metric_per_row(train_scores, save_dir=train_mean_mse_dir, subcolumn='Mean MSE', title='Training Scores - Mean MSE')
        save_metric_per_row(train_scores, save_dir=train_std_mse_dir, subcolumn='Std MSE', title='Training Scores - Std MSE')


    middle_metric_dir = f'{train_mean_mse_dir}' if not neural_network else f'{test_mae_dir}'

    # Define directories for MSE images
    mse_test_images_dirs = {'All Models': test_mse_dir}

    middle_metric_dirs = {'All Models': middle_metric_dir}

    # Define directory for MAPE images
    mape_images_dirs = {'All Models': test_mape_dir}

    # Define directory for MAE images
    mae_images_dirs = {'All Models': test_mae_dir}

    # Create PowerPoint presentation with the results
    slides_dir = f'slides/{map_name}'

    create_simple_slides(mse_test_images_dirs, middle_metric_dirs, mape_images_dirs, target_columns,
                         f'{slides_dir}/map2_all_models_metrics_results.pptx' if not neural_network else f'{slides_dir}/map2_neural_network_metrics_results.pptx')


# Executando pipeline por coluna

In [50]:
def create_final_df(results_dict, metrics):
    """
    Creates a final DataFrame by consolidating results for multiple targets and models.

    Parameters:
    - results_dict: Dictionary containing results for different targets and models.
    - metrics: List of performance metrics.

    Returns:
    - final_df: Consolidated DataFrame with results.
    """
    # Convert the dictionary to a DataFrame with models as columns and targets as rows
    consolidated_df = pd.DataFrame(results_dict).T

    # Sort columns alphabetically for better readability
    consolidated_df = consolidated_df.sort_index(axis=1)

    # Initialize an empty DataFrame for the final results
    final_df = pd.DataFrame()

    # Iterate over each model and its corresponding results
    for model_name, model_results in consolidated_df.items():
        # Convert the model's results to a DataFrame with metrics as columns
        model_df = pd.DataFrame(model_results.values.tolist(), index=model_results.index, columns=metrics)

        # Create a MultiIndex for columns with the model's name and metric names
        model_df.columns = pd.MultiIndex.from_product([[model_name], model_df.columns])

        # Concatenate the model's DataFrame to the final DataFrame
        final_df = pd.concat([final_df, model_df], axis=1)

    return final_df


def process_multiple_targets_pipeline(filepath, grid=False, neural_network=False, data_split_method='random', part_to_split=10):
    """
    Processes a pipeline for multiple target columns and models, including training and evaluation.

    Parameters:
    - filepath: Path to the dataset file.
    - grid: Boolean indicating whether to perform a grid search for hyperparameter tuning.
    - data_split_method: Method for splitting the data (default: 'random').
    - part_to_split: Number of parts to split the data if using 'random' split method.

    Returns:
    - final_df: Consolidated DataFrame with evaluation results.
    - training_scores_df: DataFrame with training scores (Mean MSE and Std MSE).
    """

    # Load data from the specified file
    data = load_data(filepath)

    # Extract coordinates from the data
    coords = data[['latitude', 'longitude']]

    # Filter variables to separate bands and nutrients
    bands, nutrients = filter_variables_by_name(data, exclude_coordinates=True)

    target_columns = ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'B', 'Cu', 'Mn', 'Zn', 'Mo', 'Ni']

    if neural_network:
        print("Neural Network Mode")

        # Train and evaluate models for all target nutrients using the pipeline
        test_scores, train_scores = pipeline(bands, nutrients[target_columns], coords, bands.columns, [], grid, neural_network, data_split_method, part_to_split)

        # Save the results
        save_results(df=test_scores, data_split_method=data_split_method, part_to_split=part_to_split, grid=grid, neural_network=neural_network)
        create_files_results(train_scores=train_scores, test_scores=test_scores, target_columns=target_columns, neural_network=True)

        return test_scores, train_scores
    else:
        test_dict = {}
        train_dict = {}

        # Iterate over each target nutrient
        for nutrient_name, nutrient_values in nutrients.items():
            print(f'Processing {nutrient_name}')

            # Train and evaluate models for the target nutrient using the pipeline
            test_scores, train_scores = pipeline(bands, nutrient_values, coords, bands.columns, [], grid, neural_network, data_split_method, part_to_split)

            # Add results to the dictionaries
            test_dict[nutrient_name] = test_scores
            train_dict[nutrient_name] = train_scores

        # Create a consolidated DataFrame with the evaluation results for test data
        final_test_scores = create_final_df(test_dict, metrics=["MSE", "MAE", "R2", "MAPE"])

        # Create a consolidated DataFrame with the evaluation results for train data
        final_train_scores = create_final_df(train_dict, metrics=["Mean MSE", "Std MSE"])

        # Save the results
        save_results(final_test_scores, data_split_method, part_to_split, grid, neural_network)
        create_files_results(final_train_scores, final_test_scores, target_columns)

        return final_test_scores, final_train_scores

# Teste do Pipeline

In [51]:
# file_path = f'data/{map_name}/interpolation/universal_interpolated_df.csv'
file_path = f'data/{map_name}/interpolation/nearest_neighbors_interpolation_df.csv'


test_scores, train_scores = process_multiple_targets_pipeline(file_path, grid=True, data_split_method='slice', neural_network = False, part_to_split=5)
test_scores

Processing N
Training models
Training LinearRegression


Best parameters of LinearRegression are: {'dim_red__n_components': 5}
Best score of LinearRegression is: -36.368761925828956

Mean CVs score: -36.368761925828956
Stds CVs score: 14.231244368702678


Evaluating models
Done
Model LinearRegression saved at models/map2/slice/N/LinearRegression_2024-02-26_20-50-36.pkl
Processing P
Training models
Training LinearRegression
Best parameters of LinearRegression are: {'dim_red__n_components': 5}
Best score of LinearRegression is: -0.18116807148614697

Mean CVs score: -0.18116807148614697
Stds CVs score: 0.06607053671532523


Evaluating models
Done
Model LinearRegression saved at models/map2/slice/P/LinearRegression_2024-02-26_20-50-36.pkl
Processing K
Training models
Training LinearRegression
Best parameters of LinearRegression are: {'dim_red__n_components': 5}
Best score of LinearRegression is: -2.8796251104957387

Mean CVs score: -2.8796251104957387
Stds CVs score: 1.8709924239598905


Evaluating models
Done
Model LinearRegression saved at mod

LinearRegression                                
                     MSE        MAE         R2      MAPE
N               7.846987   2.171275  -0.073502  0.046691
P               0.053620   0.193018  -1.886341  0.038095
K               2.117567   1.244338  -0.112259  0.078454
Ca              0.162905   0.359027   0.196101  0.046494
Mg              0.009885   0.083397   0.000345  0.031948
S               0.008937   0.090046   0.181476  0.049586
B               5.230432   1.964877   0.146227  0.061521
Cu              0.168328   0.326173  -0.873944  0.037604
Fe           1237.785482  32.330660 -31.095491  0.356649
Mn             33.007310   3.729012  -0.370195  0.043073
Zn              1.878042   1.021392  -0.314196  0.025155
Mo              0.145119   0.328671  -0.044458  0.344111
Ni              0.098936   0.285389  -0.034946  0.172013
Al            500.453480  19.950799 -17.663002  0.213056
Se              0.299516   0.439183   0.172267  0.119997
Si           7244.185800  71.270568   0.129722  0.067168
Na             21.654577   3.617777  -3.792717  0.300568
Va              0.009481   0.085773 -11.788359  0.730510
DRIS_N          0.034941   0.152105  -0.302388  1.536869
DRIS_P          0.043602   0.152772  -1.557492  0.075952
DRIS_K          0.148834   0.321050   0.117397  1.097708
DRIS_Ca         0.070278   0.222516  -0.277130  5.266996
DRIS_Mg         0.041546   0.164752   0.056570  3.606287
DRIS_S          0.138078   0.306523  -1.776917  0.329902
DRIS_B          0.133114   0.329744  -0.339688  4.442314
DRIS_Cu         0.065154   0.197798  -1.057182  0.079794
DRIS_Fe         1.408369   1.078320 -29.485988  1.696622
DRIS_Mn         0.037345   0.116311  -0.069847  0.082050
DRIS_Zn         0.036403   0.147200  -2.644688  0.219285
IMS             0.014082   0.088527 -24.223693  0.170369
IBN             4.251490   1.922013  -8.513940  0.184392

In [52]:
# file_path = f'data/{map_name}/interpolation/universal_interpolated_df.csv'
file_path = f'data/{map_name}/interpolation/nearest_neighbors_interpolation_df.csv'

test_scores, train_scores = process_multiple_targets_pipeline(file_path, grid=True, neural_network = True, data_split_method='slice', part_to_split=5)
test_scores

Neural Network Mode
Training models
Training NeuralNetwork
Best parameters of NeuralNetwork are: {'model__batch_size': 32, 'model__epochs': 100, 'model__input_neurons': 5, 'model__loss': 'mse', 'model__optimizer': 'adam', 'model__output_neurons': 12}
Best score of NeuralNetwork is: -9.786057050517439

Mean CVs score: -9.786057050517439
Stds CVs score: 2.090294836745772


Evaluating models
Done
INFO:tensorflow:Assets written to: C:\Users\Lucas\AppData\Local\Temp\tmp6ub3gfta\assets


INFO:tensorflow:Assets written to: C:\Users\Lucas\AppData\Local\Temp\tmp6ub3gfta\assets


Model NeuralNetwork saved at models/map2/neural_network/['N' 'P' 'K' 'Ca' 'Mg' 'S' 'B' 'Cu' 'Mn' 'Zn' 'Mo' 'Ni']/NeuralNetwork_2024-02-26_21-02-00.pkl
Results saved at data/map2/prediction_results/neural_network/neural_network_2024-02-26_21-02-01_part_splited_5_results.xlsx


NeuralNetwork                              
             MSE       MAE        R2      MAPE
N       8.319985  2.178986 -0.138210  0.046714
P       0.036268  0.151055 -0.952306  0.029686
K       2.366067  1.308656 -0.242785  0.082943
Ca      0.175959  0.365230  0.131679  0.047154
Mg      0.009985  0.082274 -0.009718  0.031686
S       0.009071  0.089077  0.169213  0.049812
B       6.543761  2.102554 -0.068150  0.065207
Cu      0.204683  0.365007 -1.278675  0.042103
Mn     52.217254  4.878721 -1.167636  0.056087
Zn      3.424264  1.291132 -1.396195  0.031835
Mo      0.133015  0.303577  0.042654  0.309304
Ni      0.105579  0.287687 -0.104432  0.171116

In [53]:
train_scores

{'NeuralNetwork': {'Mean MSE': 9.786057050517439,
  'Std MSE': 2.090294836745772}}